In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [3]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test_data.head(20)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
5,0027_01,Earth,False,F/7/P,TRAPPIST-1e,31.0,False,0.0,1615.0,263.0,113.0,60.0,Karlen Ricks
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.0,False,0.0,NaN,0.0,0.0,0.0,Aldah Ainserfle
7,0032_01,Europa,True,D/0/S,TRAPPIST-1e,20.0,False,0.0,0.0,0.0,0.0,0.0,Acrabi Pringry
8,0032_02,Europa,True,D/0/S,55 Cancri e,23.0,False,0.0,0.0,0.0,0.0,0.0,Dhena Pringry
9,0033_01,Earth,False,F/7/S,55 Cancri e,24.0,False,0.0,639.0,0.0,0.0,0.0,Eliana Delazarson


In [4]:
train_data.head(20)
test_data.isnull().sum()


PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [5]:
clean_data = train_data.copy()
clean_data_test = test_data.copy()
# Le nom des colonnes que je veux garder dans mon dataset
features_data = ["Transported","PassengerId","HomePlanet","CryoSleep","Destination","Age","VIP"]
test_features_data = ["PassengerId","HomePlanet","CryoSleep","Destination","Age","VIP"]

# Je crée mon dataset avec mes features
clean_train_data = clean_data[features_data]
clean_test_data = clean_data_test[test_features_data]

# je retire les valeurs NaN
clean_train_data = clean_train_data.fillna(method='bfill', axis=0).fillna(0)
clean_test_data = clean_test_data.fillna(method='bfill', axis=0).fillna(0)

# je verifie s'il reste de NaN
clean_train_data.isnull().sum()
clean_test_data.isnull().sum()

# je check si il reste assez de données pour faire ma regression



PassengerId    0
HomePlanet     0
CryoSleep      0
Destination    0
Age            0
VIP            0
dtype: int64

In [6]:
# Je crée ma target
y = clean_train_data["Transported"]
y = y.map({True: 1, False : 0})

features = ["HomePlanet","CryoSleep","Destination","Age","VIP"]

# j'ai retiré passengId car je n'ai pas trouvé l'erreur
# tester avec la colonne "cabin" si je trouve une solution (ajouter 3 colonnes distinct par exemple)

# Je crée mes features dans la variable X_train
X_train = clean_train_data[features]
X_test = clean_test_data[features]

# J'utilise get dummies pour transformer les str en int avec les features que je veux garder
X_train = pd.get_dummies(X_train[features])
X_test = pd.get_dummies(X_test[features])

In [7]:
#Utilisation du train_test_split pour train
train_X, val_X, train_y, val_y = train_test_split(X_train, y, random_state=1)

train_model = RandomForestClassifier(random_state=1, max_depth = 5, n_estimators = 100)

train_model.fit(train_X, train_y)

train_predict = train_model.predict(val_X)

train_error = mean_absolute_error(train_predict, val_y)
print("Mon taux d'erreur est de :", train_error)

Mon taux d'erreur est de : 0.2667893284268629


In [8]:
# Faire le code pour mon data_test et submit
test_model = RandomForestClassifier(random_state=1, max_depth = 5, n_estimators = 100)

test_model.fit(X_train,y)

test_predict = test_model.predict(X_test)
test_predict = test_predict.astype(bool)
print(test_predict)


[ True False  True ...  True False  True]


In [9]:
output = pd.DataFrame({'PassengerId': clean_test_data.PassengerId,
                       "Transported": test_predict})
output.to_csv('submission.csv', index=False)